# Create Data Set

In [162]:
%matplotlib inline
import matplotlib.pyplot as plt
import imageio
import numpy as np
import random
import os

## Crop images - 81x81x3

In [163]:
def cropped_traffic_lights(original_img, label_img, data, labels):    
    mask = ((label_img[:,:,0]==250) & (label_img[:,:,1]==170)  & (label_img[:,:,2]==30))
    Ti, Tj = np.where(mask == True)
    Fi, Fj = np.where(mask == False)
    size = 40

    for i in range(5):
        try:
            rT=random.randint(0,len(Ti))
            rF=random.randint(0,len(Fi))
            
            Tx ,Ty = Ti[rT],Tj[rT]
            Fx, Fy = Fi[rF],Fj[rF]
            
            tl_cropped = original_img[Tx-size:Tx+size+1,Ty-size:Ty+size+1, :]
            np.ndarray.tofile(tl_cropped, data)
            labels.write(b'1')
            
            ntl_cropped = original_img[Fx-size:Fx+size+1,Fy-size:Fy+size+1, :] 
            np.ndarray.tofile(ntl_cropped, data)
            labels.write(b'0')
            
        except:
            break

## Laod TRAIN Directories images, crop and save to binary file

In [164]:
label_dir = r'C:\Users\USER\BootCamp\Python\Mobileye Project\CityScapes\gtFine\train'
img_dir = r'C:\Users\USER\BootCamp\Python\Mobileye Project\CityScapes\leftImg8bit\train'

In [170]:
images = []
for r, d, f in os.walk(img_dir):
    for img in f:
        if '.png' in img:
            images.append(os.path.join(r, img))         

In [172]:
with open('data.bin', 'ab') as data:
    with open('labels.bin', 'ab') as labels:
        for i in range(len(images)):
            color_path = images[i].replace(img_dir,"").replace("_leftImg8bit.png","_gtFine_color.png")
            label_img = imageio.imread(label_dir + color_path)
            original_img = imageio.imread(files[i])
            cropped_traffic_lights(original_img, label_img, data, labels)
            if i % 100 == 0:
                print(f"logging... {i} / {len(images)}")
        print("end")

logging... 0 / 2975
logging... 100 / 2975
logging... 200 / 2975
logging... 300 / 2975
logging... 400 / 2975
logging... 500 / 2975
logging... 600 / 2975
logging... 700 / 2975
logging... 800 / 2975
logging... 900 / 2975
logging... 1000 / 2975
logging... 1100 / 2975
logging... 1200 / 2975
logging... 1300 / 2975
logging... 1400 / 2975
logging... 1500 / 2975
logging... 1600 / 2975
logging... 1700 / 2975
logging... 1800 / 2975
logging... 1900 / 2975
logging... 2000 / 2975
logging... 2100 / 2975
logging... 2200 / 2975
logging... 2300 / 2975
logging... 2400 / 2975
logging... 2500 / 2975
logging... 2600 / 2975
logging... 2700 / 2975
logging... 2800 / 2975
logging... 2900 / 2975


## Do the saim for VAL

In [173]:
label_dir = r'C:\Users\USER\BootCamp\Python\Mobileye Project\CityScapes\gtFine\val'
img_dir = r'C:\Users\USER\BootCamp\Python\Mobileye Project\CityScapes\leftImg8bit\val'

images = []
for r, d, f in os.walk(img_dir):
    for img in f:
        if '.png' in img:
            images.append(os.path.join(r, img))  
            
with open('data.bin', 'ab') as data:
    with open('labels.bin', 'ab') as labels:
        for i in range(len(images)):
            color_path = images[i].replace(img_dir,"").replace("_leftImg8bit.png","_gtFine_color.png")
            label_img = imageio.imread(label_dir + color_path)
            original_img = imageio.imread(files[i])
            cropped_traffic_lights(original_img, label_img, data, labels)
            if i % 100 == 0:
                print(f"logging... {i} / {len(images)}")
        print("end")            

logging... 0 / 500
logging... 100 / 500
logging... 200 / 500
logging... 300 / 500
logging... 400 / 500
end
